In [1]:
# Import the dependencies.
import pandas as pd
# os for opening file
import os
# gmaps for google maps api
import gmaps
import requests
# API key
from config import g_key

In [2]:
# Save file path to variable
file_to_load = os.path.join("Resources","WeatherPy_challenge.csv")

In [3]:
# Create DataFrme from csv
vacation_df = pd.read_csv(file_to_load)

In [4]:
# Check df was properly imported from csv
vacation_df.head(1)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
0,0,Busselton,AU,2020-04-01 23:00:27,-33.65,115.33,57.99,67,94,11.52,overcast clouds,Clouds,0.0,0.0


In [5]:
# Explore the data, check data types
vacation_df.dtypes

Unnamed: 0                int64
City                     object
Country                  object
Date                     object
Lat                     float64
Lng                     float64
Max Temp                float64
Humidity                  int64
Cloudiness                int64
Wind Speed              float64
Current Weather          object
Type of Weather          object
Rain Inches(last 3h)    float64
Snow Inches(last 3h)    float64
dtype: object

In [6]:
# Explore the data, check for null values
vacation_df.notnull().sum()

Unnamed: 0              575
City                    575
Country                 573
Date                    575
Lat                     575
Lng                     575
Max Temp                575
Humidity                575
Cloudiness              575
Wind Speed              575
Current Weather         575
Type of Weather         575
Rain Inches(last 3h)    575
Snow Inches(last 3h)    575
dtype: int64

In [7]:
# Drop rows with the two nan values in the dataset
vacation_df = vacation_df.dropna()

In [8]:
# Confirm rows were dropped
vacation_df.notnull().sum()

Unnamed: 0              573
City                    573
Country                 573
Date                    573
Lat                     573
Lng                     573
Max Temp                573
Humidity                573
Cloudiness              573
Wind Speed              573
Current Weather         573
Type of Weather         573
Rain Inches(last 3h)    573
Snow Inches(last 3h)    573
dtype: int64

In [9]:
# Get users minimum and maximum temperatures for their trip
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 60


In [10]:
# Function that takes either yes or no from user
def get_choice(choices, precip_option):
  choice = ""
  while choice not in choices:
      choice = input(f"Would you like it to be {precip_option} on your vacation? [%s]:" % ", ".join(choices))
  return choice
rain_option = get_choice(["yes","no"], "raining")
snow_option = get_choice(["yes","no"], "snowing")

Would you like it to be raining on your vacation? [yes, no]: no
Would you like it to be snowing on your vacation? [yes, no]: yes


In [11]:

# Get the locations with temperatures within user defined params
vacation_temp_df = vacation_df.loc[(vacation_df['Max Temp'] >= min_temp)&(vacation_df['Max Temp'] <=max_temp)]

In [12]:
vacation_temp_df.count()

Unnamed: 0              212
City                    212
Country                 212
Date                    212
Lat                     212
Lng                     212
Max Temp                212
Humidity                212
Cloudiness              212
Wind Speed              212
Current Weather         212
Type of Weather         212
Rain Inches(last 3h)    212
Snow Inches(last 3h)    212
dtype: int64

In [13]:
# Set variables to be passed to loc 
rain = "Rain"
snow = "Snow"
if rain_option == "yes":
    vacation_weather_df_1 = vacation_temp_df.loc[(vacation_temp_df['Type of Weather'] == rain)] 
else:
    vacation_weather_df_1 = vacation_temp_df.loc[(vacation_temp_df['Type of Weather'] != rain)]

if snow_option == "yes":
    vacation_weather_df_2 = vacation_weather_df_1.loc[(vacation_weather_df_1['Type of Weather']== snow)]
else:
     vacation_weather_df_2 = vacation_weather_df_1.loc[(vacation_weather_df_1['Type of Weather']!= snow)]
        


In [14]:
vacation_weather_df_2.count()

Unnamed: 0              17
City                    17
Country                 17
Date                    17
Lat                     17
Lng                     17
Max Temp                17
Humidity                17
Cloudiness              17
Wind Speed              17
Current Weather         17
Type of Weather         17
Rain Inches(last 3h)    17
Snow Inches(last 3h)    17
dtype: int64

In [15]:
vacation_weather_df_2.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
19,19,Ostrovnoy,RU,2020-04-01 23:00:33,68.05,39.51,20.75,96,91,9.15,light snow,Snow,0.0,0.010630
23,23,Severo-Kurilsk,RU,2020-04-01 23:00:35,50.68,156.12,34.63,91,100,4.27,light snow,Snow,0.0,0.004724
56,56,Provideniya,RU,2020-04-01 23:00:44,64.38,-173.30,26.60,86,90,6.71,snow,Snow,0.0,0.000000
81,81,Yar-Sale,RU,2020-04-01 23:00:52,66.83,70.83,22.60,97,100,26.80,snow,Snow,0.0,0.086614
136,136,Dzhebariki-Khaya,RU,2020-04-01 23:01:08,62.22,135.80,23.05,95,100,5.55,light snow,Snow,0.0,0.024803


In [16]:
hotel_df = vacation_weather_df_2[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()

In [17]:
hotel_df.head(3)

,City,Country,Max Temp,Lat,Lng
19,Ostrovnoy,RU,20.75,68.05,39.51
23,Severo-Kurilsk,RU,34.63,50.68,156.12
56,Provideniya,RU,26.60,64.38,-173.30


In [18]:
params = {'radius': 5000, 'type': 'lodging', 'key': g_key}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("fin")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
fin


In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
# Store the DataFrame Row.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
19,Ostrovnoy,RU,20.75,68.05,39.51,NaN
23,Severo-Kurilsk,RU,34.63,50.68,156.12,Kak Doma
56,Provideniya,RU,26.60,64.38,-173.30,NaN
81,Yar-Sale,RU,22.60,66.83,70.83,Yalemd
136,Dzhebariki-Khaya,RU,23.05,62.22,135.80,NaN
142,Sobolevo,RU,33.98,54.43,31.90,NaN
259,Homer,US,39.00,59.64,-151.55,Beluga Lake Lodge
296,Lavrentiya,RU,23.00,65.58,-171.00,NaN
318,Nome,US,23.00,64.50,-165.41,Dredge 7 Inn
364,Okhotsk,RU,21.87,59.38,143.30,NaN


In [23]:
save_path = os.path.join('Resources','WeatherPy_vacation.csv')
hotel_df.to_csv(save_path)